In [1]:
import sys
import os
import toml

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np
import scipy.optimize as SciOpt
from scipy.special import rel_entr
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tqdm.notebook import tqdm
from copy import deepcopy

sys.path.append(os.path.join(sys.path[0], '../..'))

from data.io import Reader
from data.data_generators import DataGenerator
from lsm.lsmtype import Policy
import lsm.cost as CostFunc
import lsm.solver as Solvers

In [2]:
config = Reader.read_config('../../endure.toml')
data_gen = DataGenerator(config)
solver = Solvers.EndureQSolver(config)
min_kwargs = {
    'method': 'SLSQP',
    'options': {'ftol': 1e-12, 'disp': False}
}

In [3]:
ewls = config['workloads']
rhos = np.arange(0, 4, 0.25)
ewls

[{'id': 0, 'z0': 0.25, 'z1': 0.25, 'q': 0.25, 'w': 0.25},
 {'id': 1, 'z0': 0.97, 'z1': 0.01, 'q': 0.01, 'w': 0.01},
 {'id': 2, 'z0': 0.01, 'z1': 0.97, 'q': 0.01, 'w': 0.01},
 {'id': 3, 'z0': 0.01, 'z1': 0.01, 'q': 0.97, 'w': 0.01},
 {'id': 4, 'z0': 0.01, 'z1': 0.01, 'q': 0.01, 'w': 0.97},
 {'id': 5, 'z0': 0.49, 'z1': 0.49, 'q': 0.01, 'w': 0.01},
 {'id': 6, 'z0': 0.49, 'z1': 0.01, 'q': 0.49, 'w': 0.01},
 {'id': 7, 'z0': 0.49, 'z1': 0.01, 'q': 0.01, 'w': 0.49},
 {'id': 8, 'z0': 0.01, 'z1': 0.49, 'q': 0.49, 'w': 0.01},
 {'id': 9, 'z0': 0.01, 'z1': 0.49, 'q': 0.01, 'w': 0.49},
 {'id': 10, 'z0': 0.01, 'z1': 0.01, 'q': 0.49, 'w': 0.49},
 {'id': 11, 'z0': 0.33, 'z1': 0.33, 'q': 0.33, 'w': 0.01},
 {'id': 12, 'z0': 0.33, 'z1': 0.33, 'q': 0.01, 'w': 0.33},
 {'id': 13, 'z0': 0.33, 'z1': 0.01, 'q': 0.33, 'w': 0.33},
 {'id': 14, 'z0': 0.01, 'z1': 0.33, 'q': 0.33, 'w': 0.33}]

In [4]:
def sample_workloads(samples=10000):
    wls = []
    for _ in range(samples):
        wls.append(data_gen._sample_workload(4))
    return wls

In [5]:
def create_tunings():
    df = []
    pbar = tqdm(ewls, desc='Workloads')

    for idx, w in enumerate(pbar):
        pbar.set_postfix(workload=w)
        row = {}
        row['workload_idx'] = idx
        z0, z1, q, w = (w['z0'], w['z1'], w['q'], w['w'])
        row['z0'], row['z1'], row['q'], row['w'] = (z0, z1, q, w)

        nominal = solver.find_nominal_design(z0, z1, q, w, minimizer_kwargs=min_kwargs)
        row['nominal_h'] = h_init = nominal.x[0]
        row['nominal_T'] = T_init = nominal.x[1]
        row['nominal_Q'] = Q_init = nominal.x[2]
        row['nominal_cost'] = nominal.fun
        row['nominal_msg'] = nominal.message
        for rho in rhos:
            row['rho'] = rho
            robust = solver.find_robust_design(rho, z0, z1, q, w, [h_init, T_init, Q_init], minimizer_kwargs=min_kwargs)
            row['robust_h'] = robust.x[0]
            row['robust_T'] = robust.x[1]
            row['robust_Q'] = robust.x[2]
            row['robust_msg'] = robust.message
            row['robust_cost'] = solver.nominal_objective(robust.x[0:3], z0, z1, q, w)

            # Append the design to the dataframe
            df.append(deepcopy(row))

    df = pd.DataFrame(df)
    return df

In [6]:
tunings = create_tunings()
tunings

Workloads:   0%|          | 0/15 [00:00<?, ?it/s]

/Users/ndhuynh/miniforge3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:284: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/ndhuynh/repos/endure/endure-torch/notebook/data-gen/../../lsm/solver.py:24: RuntimeWarning: overflow encountered in exp
  return np.exp(input) - 1


,workload_idx,z0,z1,q,w,nominal_h,nominal_T,nominal_Q,nominal_cost,nominal_msg,rho,robust_h,robust_T,robust_Q,robust_msg,robust_cost
0,0,0.25,0.25,0.25,0.25,3.30,6.18,1.07,2.83,Optimization terminated successfully,0.00,3.30,6.19,1.07,Optimization terminated successfully,2.83
1,0,0.25,0.25,0.25,0.25,3.30,6.18,1.07,2.83,Optimization terminated successfully,0.25,0.90,6.42,1.23,Optimization terminated successfully,3.01
2,0,0.25,0.25,0.25,0.25,3.30,6.18,1.07,2.83,Optimization terminated successfully,0.50,0.00,6.55,1.28,Optimization terminated successfully,3.22
3,0,0.25,0.25,0.25,0.25,3.30,6.18,1.07,2.83,Optimization terminated successfully,0.75,0.00,6.68,1.32,Optimization terminated successfully,3.25
4,0,0.25,0.25,0.25,0.25,3.30,6.18,1.07,2.83,Optimization terminated successfully,1.00,0.00,6.68,1.32,Optimization terminated successfully,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,14,0.01,0.33,0.33,0.33,0.06,5.93,1.00,3.52,Optimization terminated successfully,2.75,0.00,6.68,1.32,Optimization terminated successfully,3.59
236,14,0.01,0.33,0.33,0.33,0.06,5.93,1.00,3.52,Optimization terminated successfully,3.00,0.00,6.68,1.32,Optimization terminated successfully,3.59
237,14,0.01,0.33,0.33,0.33,0.06,5.93,1.00,3.52,Optimization terminated successfully,3.25,0.00,6.68,1.32,Optimization terminated successfully,3.59
238,14,0.01,0.33,0.33,0.33,0.06,5.93,1.00,3.52,Optimization terminated successfully,3.50,0.00,6.68,1.32,Optimization terminated successfully,3.59


In [7]:
tunings['robust_msg'].unique()

array(['Optimization terminated successfully',
       'Inequality constraints incompatible'], dtype=object)

In [8]:
samples = sample_workloads()
distances = {}
for idx, wl in enumerate(ewls):
    key = idx
    w0 = [wl['z0'], wl['z1'], wl['q'], wl['w']]
    distances[key] = []
    for sample in samples:
        distances[key].append(np.sum(rel_entr(sample, w0)))

In [9]:
comparisons = []
for tuning in tqdm(tunings.to_records('dict'), desc='Tunings'):
    row = {}
    row['workload_idx'] = tuning['workload_idx']
    row['z0'] = tuning['z0']
    row['z1'] = tuning['z1']
    row['q'] = tuning['q']
    row['w'] = tuning['w']
    row['robust_rho'] = tuning['rho']
    row['robust_h'] = tuning['robust_h']
    row['robust_T'] = tuning['robust_T']
    row['robust_Q'] = tuning['robust_Q']

    row['nominal_h'] = tuning['nominal_h']
    row['nominal_T'] = tuning['nominal_T']
    row['nominal_Q'] = tuning['nominal_Q']
    nominal = [row['nominal_h'], row['nominal_T'], row['nominal_Q']]
    robust = [row['robust_h'], row['robust_T'], row['robust_Q']]

    distance = distances[row['workload_idx']]
    for idx, w_hat in enumerate(samples):
        row['rho_hat'] = distance[idx]
        row['z0_hat'] = w_hat[0]
        row['z1_hat'] = w_hat[1]
        row['q_hat'] = w_hat[2]
        row['w_hat'] = w_hat[3]
        row['sample_idx'] = idx
        row['nominal_cost'] = solver.nominal_objective(nominal, *w_hat)
        row['robust_cost'] = solver.nominal_objective(robust, *w_hat)
        comparisons.append(deepcopy(row))

df = pd.DataFrame(comparisons)

Tunings:   0%|          | 0/240 [00:00<?, ?it/s]

In [10]:
df.to_csv('/Users/ndhuynh/sandbox/data/exp01.csv', index=False)